In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_csv(r'C:\Users\aa\PyCharmMiscProject\Tiktok_analysis\tiktok_dataset.csv', sep=';')

In [4]:
data.head()

,#,claim_status,video_id,video_duration_sec,video_transcription_text,verified_status,author_ban_status,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
0,1,claim,7017666017,59,someone shared with me that drone deliveries a...,not verified,under review,343296.0,19425.0,241.0,1.0,0.0
1,2,claim,4014381136,32,someone shared with me that there are more mic...,not verified,active,140877.0,77355.0,19034.0,1161.0,684.0
2,3,claim,9859838091,31,someone shared with me that american industria...,not verified,active,902185.0,97690.0,2858.0,833.0,329.0
3,4,claim,1866847991,25,someone shared with me that the metro of st. p...,not verified,active,437506.0,239954.0,34812.0,1234.0,584.0
4,5,claim,7105231098,19,someone shared with me that the number of busi...,not verified,active,56167.0,34987.0,4110.0,547.0,152.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19382 entries, 0 to 19381
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   #                         19382 non-null  int64  
 1   claim_status              19084 non-null  object 
 2   video_id                  19382 non-null  int64  
 3   video_duration_sec        19382 non-null  int64  
 4   video_transcription_text  19084 non-null  object 
 5   verified_status           19382 non-null  object 
 6   author_ban_status         19382 non-null  object 
 7   video_view_count          19084 non-null  float64
 8   video_like_count          19084 non-null  float64
 9   video_share_count         19084 non-null  float64
 10  video_download_count      19084 non-null  float64
 11  video_comment_count       19084 non-null  float64
dtypes: float64(5), int64(3), object(4)
memory usage: 1.8+ MB


In [6]:
data.describe()

,#,video_id,video_duration_sec,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
count,19382.000000,1.938200e+04,19382.000000,19084.000000,19084.000000,19084.000000,19084.000000,19084.000000
mean,9691.500000,5.627454e+09,32.421732,254708.558688,84304.636030,16735.248323,1049.429627,349.312146
std,5595.245794,2.536440e+09,16.229967,322893.280814,133420.546814,32036.174350,2004.299894,799.638865
min,1.000000,1.234959e+09,5.000000,20.000000,0.000000,0.000000,0.000000,0.000000
25%,4846.250000,3.430417e+09,18.000000,4942.500000,810.750000,115.000000,7.000000,1.000000
50%,9691.500000,5.618664e+09,32.000000,9954.500000,3403.500000,717.000000,46.000000,9.000000
75%,14536.750000,7.843960e+09,47.000000,504327.000000,125020.000000,18222.000000,1156.250000,292.000000
max,19382.000000,9.999873e+09,60.000000,999817.000000,657830.000000,256130.000000,14994.000000,9599.000000


In [7]:
data.shape

(19382, 12)

In [8]:
data.isnull().sum()

#                             0
claim_status                298
video_id                      0
video_duration_sec            0
video_transcription_text    298
verified_status               0
author_ban_status             0
video_view_count            298
video_like_count            298
video_share_count           298
video_download_count        298
video_comment_count         298
dtype: int64

In [11]:
data['claim_status'].value_counts()

claim_status
claim      9608
opinion    9476
Name: count, dtype: int64

Data Preprocessing

	Identify missing values (NaN) in the dataset.

	Decide on a strategy to handle them (e.g., filling with mean/median/mode, forward/backward filling, or dropping rows/columns).


In [12]:
null_values = data.isnull()
print(null_values)

           #  claim_status  video_id  video_duration_sec  \
0      False         False     False               False   
1      False         False     False               False   
2      False         False     False               False   
3      False         False     False               False   
4      False         False     False               False   
...      ...           ...       ...                 ...   
19377  False          True     False               False   
19378  False          True     False               False   
19379  False          True     False               False   
19380  False          True     False               False   
19381  False          True     False               False   

       video_transcription_text  verified_status  author_ban_status  \
0                         False            False              False   
1                         False            False              False   
2                         False            False              Fals

In [13]:
null_count = data.isnull().sum()
print(null_count)

#                             0
claim_status                298
video_id                      0
video_duration_sec            0
video_transcription_text    298
verified_status               0
author_ban_status             0
video_view_count            298
video_like_count            298
video_share_count           298
video_download_count        298
video_comment_count         298
dtype: int64


In [15]:
missing_percentage = data.isnull().sum() / len(data) * 100
print(missing_percentage)

#                           0.0
claim_status                0.0
video_id                    0.0
video_duration_sec          0.0
video_transcription_text    0.0
verified_status             0.0
author_ban_status           0.0
video_view_count            0.0
video_like_count            0.0
video_share_count           0.0
video_download_count        0.0
video_comment_count         0.0
dtype: float64


In [16]:
null_summary = data.isnull().sum().reset_index()
null_summary.columns = ['Column', 'MissingValues']
null_summary['Percentage'] = (null_summary['MissingValues'] / len(data)) * 100
print(null_summary)

                      Column  MissingValues  Percentage
0                          #              0         0.0
1               claim_status              0         0.0
2                   video_id              0         0.0
3         video_duration_sec              0         0.0
4   video_transcription_text              0         0.0
5            verified_status              0         0.0
6          author_ban_status              0         0.0
7           video_view_count              0         0.0
8           video_like_count              0         0.0
9          video_share_count              0         0.0
10      video_download_count              0         0.0
11       video_comment_count              0         0.0


In [27]:
# Calculate the IQR for numerical columns
numerical_columns = ['video_view_count', 'video_like_count', 'video_share_count', 'video_download_count']
Q1 = data[numerical_columns].quantile(0.25)
Q3 = data[numerical_columns].quantile(0.75)
IQR = Q3 - Q1

# Determine the outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter rows within the outlier boundaries
print("Shape before removing outliers:", data.shape)
data = data[~((data[numerical_columns] < lower_bound) | (data[numerical_columns] > upper_bound)).any(axis=1)]
print("Shape after removing outliers:", data.shape)

Shape before removing outliers: (19084, 12)
Shape after removing outliers: (15437, 12)


In [17]:
only_nulls = data[data.isnull()]
print(only_nulls)


        # claim_status  video_id  video_duration_sec video_transcription_text  \
0     NaN          NaN       NaN                 NaN                      NaN   
1     NaN          NaN       NaN                 NaN                      NaN   
2     NaN          NaN       NaN                 NaN                      NaN   
3     NaN          NaN       NaN                 NaN                      NaN   
4     NaN          NaN       NaN                 NaN                      NaN   
...    ..          ...       ...                 ...                      ...   
19079 NaN          NaN       NaN                 NaN                      NaN   
19080 NaN          NaN       NaN                 NaN                      NaN   
19081 NaN          NaN       NaN                 NaN                      NaN   
19082 NaN          NaN       NaN                 NaN                      NaN   
19083 NaN          NaN       NaN                 NaN                      NaN   

      verified_status autho

Total missing values by column

Rows with missing values

Percentage of missing values by column




In [19]:
print("Total missing values by column:")
print(data.isnull().sum())

print("\nRows with missing values:")
print(data[data.isnull().any(axis=1)])

print("\nPercentage of missing values by column:")
print((data.isnull().sum() / len(data)) * 100)

Total missing values by column:
#                           0
claim_status                0
video_id                    0
video_duration_sec          0
video_transcription_text    0
verified_status             0
author_ban_status           0
video_view_count            0
video_like_count            0
video_share_count           0
video_download_count        0
video_comment_count         0
dtype: int64

Rows with missing values:
Empty DataFrame
Columns: [#, claim_status, video_id, video_duration_sec, video_transcription_text, verified_status, author_ban_status, video_view_count, video_like_count, video_share_count, video_download_count, video_comment_count]
Index: []

Percentage of missing values by column:
#                           0.0
claim_status                0.0
video_id                    0.0
video_duration_sec          0.0
video_transcription_text    0.0
verified_status             0.0
author_ban_status           0.0
video_view_count            0.0
video_like_count            

In [21]:
# Drop all rows with null values
data = data.dropna()

# Print the shape of the DataFrame to confirm the operation
print("Shape of the DataFrame after dropping rows with null values:", data.shape)

# Checking rows with null values (should be empty after dropping)
rows_with_null = data[data.isnull().any(axis=1)]
print(rows_with_null)

Shape of the DataFrame after dropping rows with null values: (19084, 12)
Empty DataFrame
Columns: [#, claim_status, video_id, video_duration_sec, video_transcription_text, verified_status, author_ban_status, video_view_count, video_like_count, video_share_count, video_download_count, video_comment_count]
Index: []


In [22]:
data.isnull().sum()

#                           0
claim_status                0
video_id                    0
video_duration_sec          0
video_transcription_text    0
verified_status             0
author_ban_status           0
video_view_count            0
video_like_count            0
video_share_count           0
video_download_count        0
video_comment_count         0
dtype: int64

	Check for and remove any duplicate rows to ensure that each entry is unique.

In [23]:
data.nunique()

#                           19084
claim_status                    2
video_id                    19084
video_duration_sec             56
video_transcription_text    19012
verified_status                 2
author_ban_status               3
video_view_count            15632
video_like_count            12224
video_share_count            9231
video_download_count         4336
video_comment_count          2424
dtype: int64

In [25]:
data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
19079    False
19080    False
19081    False
19082    False
19083    False
Length: 19084, dtype: bool

In [26]:
duplicates = data[data.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [#, claim_status, video_id, video_duration_sec, video_transcription_text, verified_status, author_ban_status, video_view_count, video_like_count, video_share_count, video_download_count, video_comment_count]
Index: []


	Identify and handle outliers in numerical columns like views, likes, comments, etc., using methods such as the IQR method or Z-score.
